# Black-Scholes Options Pricing and Dynamic Hedging Analysis

This notebook demonstrates a comprehensive options pricing and hedging simulation project that covers:

1. **Black-Scholes-Merton (BSM) Model** - Closed-form pricing and Greeks
2. **Monte Carlo Simulation** - Path-dependent pricing methods
3. **Dynamic Delta Hedging** - Portfolio rebalancing strategies
4. **Stochastic Volatility** - Heston model for realistic market dynamics
5. **Transaction Costs** - Real-world trading friction effects

## Project Overview

We simulate a scenario where:
- The **true market** follows a Heston stochastic volatility model
- The **hedger** assumes constant volatility (Black-Scholes world)
- We measure the **hedging error** that results from this model mismatch

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from simulator import DynamicHedgingSimulator
from pricers import BlackScholesPricer, MonteCarloPricer

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)  # For reproducible results

## 1. Parameter Setup

Let's define our market parameters and option characteristics.

In [ ]:
# --- Option and Market Parameters ---
S0 = 100.0        # Initial stock price
K = 100.0         # Strike price (at-the-money)
T = 1.0           # Time to maturity (1 year)
r = 0.05          # Risk-free rate (5%)
option_type = "call"
num_steps = 252   # Number of rebalancing steps (daily rebalancing)
transaction_cost = 0.001  # 0.1% transaction cost (10 bps)

# True Market Parameters (Heston Model)
heston_params = {
    'v0': 0.04,     # Initial variance (σ=20%)
    'kappa': 2.0,   # Mean reversion speed
    'theta': 0.04,  # Long-term variance (σ_∞=20%)
    'xi': 0.3,      # Volatility of volatility (30%)
    'rho': -0.7     # Correlation between stock and volatility (-70%)
}

# Hedger's Belief (BSM Model)
hedge_sigma = 0.20  # Constant volatility assumption (20%)

print(f"Option Setup:")
print(f"  Stock Price: ${S0}, Strike: ${K}, Time to Expiry: {T} years")
print(f"  Risk-free Rate: {r*100:.1f}%, Transaction Cost: {transaction_cost*100:.1f}%")
print(f"\nTrue Market (Heston):")
print(f"  Initial Vol: {np.sqrt(heston_params['v0'])*100:.1f}%")
print(f"  Long-term Vol: {np.sqrt(heston_params['theta'])*100:.1f}%")
print(f"  Vol-of-vol: {heston_params['xi']*100:.1f}%")
print(f"  Correlation: {heston_params['rho']:.1f}")
print(f"\nHedger's Assumption: {hedge_sigma*100:.1f}% constant volatility")

## 2. Pricing Model Validation

Let's first validate our pricing models by comparing Black-Scholes analytical results with Monte Carlo simulation.

In [ ]:
# Initialize pricers
bs_pricer = BlackScholesPricer()
mc_pricer = MonteCarloPricer(num_sims=100000, num_steps=100)

# Calculate prices and Greeks
bs_price = bs_pricer.price(S0, K, T, r, hedge_sigma, option_type)
bs_delta = bs_pricer.delta(S0, K, T, r, hedge_sigma, option_type)
bs_gamma = bs_pricer.gamma(S0, K, T, r, hedge_sigma)
bs_vega = bs_pricer.vega(S0, K, T, r, hedge_sigma)

mc_price = mc_pricer.price(S0, K, T, r, hedge_sigma, option_type)
mc_delta = mc_pricer.delta(S0, K, T, r, hedge_sigma, option_type)
mc_gamma = mc_pricer.gamma(S0, K, T, r, hedge_sigma, option_type)
mc_vega = mc_pricer.vega(S0, K, T, r, hedge_sigma, option_type)

# Create comparison table
comparison_df = pd.DataFrame({
    'Metric': ['Price', 'Delta', 'Gamma', 'Vega'],
    'Black-Scholes': [bs_price, bs_delta, bs_gamma, bs_vega],
    'Monte Carlo': [mc_price, mc_delta, mc_gamma, mc_vega],
    'Difference (%)': [abs(bs_price - mc_price) / bs_price * 100,
                      abs(bs_delta - mc_delta) / abs(bs_delta) * 100,
                      abs(bs_gamma - mc_gamma) / bs_gamma * 100,
                      abs(bs_vega - mc_vega) / bs_vega * 100]
})

print("Pricing Model Validation:")
print(comparison_df.round(4))
print(f"\nNote: Small differences are due to Monte Carlo sampling error")

## 3. Single Path Hedging Simulation

Let's first look at a single hedging simulation to understand the mechanics.

In [ ]:
# Run a single simulation
simulator = DynamicHedgingSimulator(S0, K, T, r, option_type, transaction_cost)
hedging_pnl, s_path, pnl_path = simulator.run_simulation(hedge_sigma, heston_params, num_steps)

print(f"Single Simulation Results:")
print(f"  Final Stock Price: ${s_path[-1]:.2f}")
print(f"  Option Payoff: ${max(s_path[-1] - K, 0):.2f}")
print(f"  Hedging P&L: ${hedging_pnl:.2f}")

# Plot the simulation path
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10))
t = np.linspace(0, T, num_steps + 1)

# Stock price path
ax1.plot(t, s_path, 'b-', linewidth=2, label='Stock Price')
ax1.axhline(y=K, color='r', linestyle='--', alpha=0.7, label='Strike Price')
ax1.set_ylabel('Stock Price ($)')
ax1.set_title('Stock Price Evolution')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Portfolio P&L evolution
ax2.plot(t, pnl_path, 'g-', linewidth=2, label='Portfolio P&L')
ax2.axhline(y=0, color='k', linestyle='-', alpha=0.3)
ax2.set_ylabel('P&L ($)')
ax2.set_title('Hedging Portfolio P&L Evolution')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Delta evolution
delta_path = []
for i, (s, time) in enumerate(zip(s_path[:-1], t[:-1])):
    ttm = T - time
    if ttm > 0:
        delta = bs_pricer.delta(s, K, ttm, r, hedge_sigma, option_type)
        delta_path.append(delta)
    else:
        delta_path.append(1.0 if s > K else 0.0)

ax3.plot(t[:-1], delta_path, 'purple', linewidth=2, label='Delta')
ax3.set_ylabel('Delta')
ax3.set_xlabel('Time (Years)')
ax3.set_title('Delta Evolution (Hedge Ratio)')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Monte Carlo Hedging Analysis

Now let's run many simulations to get statistical results and measure hedging effectiveness.

In [ ]:
# Run multiple simulations
num_simulations = 1000
print(f"Running {num_simulations} simulations...")

dynamic_pnls = []
static_pnls = []
final_stock_prices = []
option_payoffs = []

simulator = DynamicHedgingSimulator(S0, K, T, r, option_type, transaction_cost)

for i in range(num_simulations):
    if (i + 1) % 100 == 0:
        print(f"  Completed {i + 1}/{num_simulations} simulations")
    
    # Dynamic hedge
    pnl_dynamic, s_path, _ = simulator.run_simulation(hedge_sigma, heston_params, num_steps)
    dynamic_pnls.append(pnl_dynamic)
    
    # Static hedge
    pnl_static = simulator.run_static_hedge(heston_params, num_steps)
    static_pnls.append(pnl_static)
    
    # Store additional metrics
    final_stock_prices.append(s_path[-1])
    option_payoffs.append(max(s_path[-1] - K, 0) if option_type == "call" else max(K - s_path[-1], 0))

# Convert to numpy arrays
dynamic_pnls = np.array(dynamic_pnls)
static_pnls = np.array(static_pnls)
final_stock_prices = np.array(final_stock_prices)
option_payoffs = np.array(option_payoffs)

print("\nSimulation completed!")

## 5. Results Analysis

Let's analyze the hedging performance and calculate key metrics.

In [ ]:
# Calculate key metrics
mean_pnl_dynamic = np.mean(dynamic_pnls)
std_pnl_dynamic = np.std(dynamic_pnls)
mean_pnl_static = np.mean(static_pnls)
std_pnl_static = np.std(static_pnls)

# Hedging error is the standard deviation of P&L
hedging_error_dynamic = std_pnl_dynamic
hedging_error_static = std_pnl_static

# Additional statistics
percentiles_dynamic = np.percentile(dynamic_pnls, [5, 25, 50, 75, 95])
percentiles_static = np.percentile(static_pnls, [5, 25, 50, 75, 95])

# Create results summary
results_df = pd.DataFrame({
    'Metric': ['Mean P&L', 'Std Dev (Hedging Error)', 'Min P&L', 'Max P&L', 
               '5th Percentile', '95th Percentile', 'Sharpe Ratio'],
    'Dynamic Hedge': [mean_pnl_dynamic, hedging_error_dynamic, 
                     np.min(dynamic_pnls), np.max(dynamic_pnls),
                     percentiles_dynamic[0], percentiles_dynamic[4],
                     mean_pnl_dynamic / hedging_error_dynamic if hedging_error_dynamic > 0 else 0],
    'Static Hedge': [mean_pnl_static, hedging_error_static,
                    np.min(static_pnls), np.max(static_pnls),
                    percentiles_static[0], percentiles_static[4],
                    mean_pnl_static / hedging_error_static if hedging_error_static > 0 else 0]
})

print("=" * 60)
print("HEDGING PERFORMANCE ANALYSIS")
print("=" * 60)
print(f"\nSimulation Parameters:")
print(f"  Number of simulations: {num_simulations:,}")
print(f"  Rebalancing frequency: Daily ({num_steps} steps)")
print(f"  Transaction cost: {transaction_cost*100:.1f}%")
print(f"\nResults Summary:")
print(results_df.round(4))

print(f"\n=" * 60)
print("KEY INSIGHTS:")
print(f"=" * 60)
improvement = (hedging_error_static - hedging_error_dynamic) / hedging_error_static * 100
print(f"• Dynamic hedging reduces risk by {improvement:.1f}%")
print(f"• Dynamic hedge error: ${hedging_error_dynamic:.2f}")
print(f"• Static hedge error: ${hedging_error_static:.2f}")
print(f"• Perfect hedge would have zero P&L variance")

## 6. Visualization of Results

Let's create comprehensive visualizations of our hedging analysis.

In [ ]:
# Create comprehensive visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. P&L Distribution Comparison
ax1.hist(dynamic_pnls, bins=50, alpha=0.7, density=True, 
         label=f'Dynamic (σ={hedging_error_dynamic:.2f})', color='blue')
ax1.hist(static_pnls, bins=50, alpha=0.7, density=True,
         label=f'Static (σ={hedging_error_static:.2f})', color='red')
ax1.axvline(0, color='black', linestyle='--', linewidth=2, alpha=0.8)
ax1.axvline(mean_pnl_dynamic, color='blue', linestyle='-', alpha=0.8)
ax1.axvline(mean_pnl_static, color='red', linestyle='-', alpha=0.8)
ax1.set_xlabel('Hedging P&L ($)')
ax1.set_ylabel('Density')
ax1.set_title('Distribution of Hedging P&L')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Q-Q Plot for normality check
from scipy import stats
stats.probplot(dynamic_pnls, dist="norm", plot=ax2)
ax2.set_title('Q-Q Plot: Dynamic Hedge P&L vs Normal Distribution')
ax2.grid(True, alpha=0.3)

# 3. P&L vs Final Stock Price
scatter = ax3.scatter(final_stock_prices, dynamic_pnls, alpha=0.6, s=20)
ax3.axhline(y=0, color='black', linestyle='--', alpha=0.8)
ax3.axvline(x=K, color='red', linestyle='--', alpha=0.8, label='Strike')
ax3.set_xlabel('Final Stock Price ($)')
ax3.set_ylabel('Dynamic Hedge P&L ($)')
ax3.set_title('Hedging P&L vs Final Stock Price')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Cumulative P&L Distribution
sorted_dynamic = np.sort(dynamic_pnls)
sorted_static = np.sort(static_pnls)
p_values = np.arange(1, len(sorted_dynamic) + 1) / len(sorted_dynamic)

ax4.plot(sorted_dynamic, p_values, label='Dynamic Hedge', linewidth=2)
ax4.plot(sorted_static, p_values, label='Static Hedge', linewidth=2)
ax4.axvline(0, color='black', linestyle='--', alpha=0.8)
ax4.set_xlabel('Hedging P&L ($)')
ax4.set_ylabel('Cumulative Probability')
ax4.set_title('Cumulative Distribution of Hedging P&L')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Sensitivity Analysis

Let's analyze how different parameters affect hedging performance.

In [ ]:
# Sensitivity analysis: Transaction costs
tc_levels = [0.0, 0.0005, 0.001, 0.002, 0.005, 0.01]
tc_results = []

print("Running transaction cost sensitivity analysis...")
for tc in tc_levels:
    sim_tc = DynamicHedgingSimulator(S0, K, T, r, option_type, tc)
    pnls_tc = []
    
    for _ in range(200):  # Smaller sample for speed
        pnl, _, _ = sim_tc.run_simulation(hedge_sigma, heston_params, num_steps)
        pnls_tc.append(pnl)
    
    hedging_error = np.std(pnls_tc)
    mean_pnl = np.mean(pnls_tc)
    tc_results.append({'TC': tc*100, 'Mean_PnL': mean_pnl, 'Hedging_Error': hedging_error})

tc_df = pd.DataFrame(tc_results)

# Plot sensitivity analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

ax1.plot(tc_df['TC'], tc_df['Hedging_Error'], 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Transaction Cost (%)')
ax1.set_ylabel('Hedging Error ($)')
ax1.set_title('Hedging Error vs Transaction Costs')
ax1.grid(True, alpha=0.3)

ax2.plot(tc_df['TC'], tc_df['Mean_PnL'], 'ro-', linewidth=2, markersize=8)
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.8)
ax2.set_xlabel('Transaction Cost (%)')
ax2.set_ylabel('Mean P&L ($)')
ax2.set_title('Mean P&L vs Transaction Costs')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nTransaction Cost Sensitivity:")
print(tc_df.round(4))

## 8. Rebalancing Frequency Analysis

How does the frequency of rebalancing affect hedging performance?

In [ ]:
# Rebalancing frequency analysis
rebal_frequencies = [21, 63, 126, 252, 504]  # Monthly, Quarterly, Semi-annual, Daily, Twice-daily
rebal_labels = ['Monthly', 'Quarterly', 'Semi-annual', 'Daily', 'Twice-daily']
rebal_results = []

print("Running rebalancing frequency analysis...")
for freq, label in zip(rebal_frequencies, rebal_labels):
    sim_freq = DynamicHedgingSimulator(S0, K, T, r, option_type, transaction_cost)
    pnls_freq = []
    
    for _ in range(200):  # Smaller sample for speed
        pnl, _, _ = sim_freq.run_simulation(hedge_sigma, heston_params, freq)
        pnls_freq.append(pnl)
    
    hedging_error = np.std(pnls_freq)
    mean_pnl = np.mean(pnls_freq)
    rebal_results.append({
        'Frequency': label, 
        'Steps': freq,
        'Mean_PnL': mean_pnl, 
        'Hedging_Error': hedging_error
    })

rebal_df = pd.DataFrame(rebal_results)

# Plot rebalancing frequency analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

ax1.plot(rebal_df['Steps'], rebal_df['Hedging_Error'], 'go-', linewidth=2, markersize=8)
ax1.set_xlabel('Rebalancing Steps per Year')
ax1.set_ylabel('Hedging Error ($)')
ax1.set_title('Hedging Error vs Rebalancing Frequency')
ax1.set_xscale('log')
ax1.grid(True, alpha=0.3)

# Add frequency labels
for i, (x, y, label) in enumerate(zip(rebal_df['Steps'], rebal_df['Hedging_Error'], rebal_df['Frequency'])):
    ax1.annotate(label, (x, y), textcoords="offset points", xytext=(0,10), ha='center')

ax2.plot(rebal_df['Steps'], rebal_df['Mean_PnL'], 'mo-', linewidth=2, markersize=8)
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.8)
ax2.set_xlabel('Rebalancing Steps per Year')
ax2.set_ylabel('Mean P&L ($)')
ax2.set_title('Mean P&L vs Rebalancing Frequency')
ax2.set_xscale('log')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nRebalancing Frequency Analysis:")
print(rebal_df.round(4))

## 9. Model Risk Analysis

What happens when the hedger's volatility assumption is wrong?

In [ ]:
# Volatility assumption analysis
vol_assumptions = np.linspace(0.10, 0.40, 10)  # 10% to 40% volatility
true_vol = np.sqrt(heston_params['v0'])  # True initial volatility
vol_results = []

print(f"Running volatility assumption analysis (true vol = {true_vol*100:.1f}%)...")

for vol_assumption in vol_assumptions:
    sim_vol = DynamicHedgingSimulator(S0, K, T, r, option_type, transaction_cost)
    pnls_vol = []
    
    for _ in range(150):  # Smaller sample for speed
        pnl, _, _ = sim_vol.run_simulation(vol_assumption, heston_params, num_steps)
        pnls_vol.append(pnl)
    
    hedging_error = np.std(pnls_vol)
    mean_pnl = np.mean(pnls_vol)
    vol_results.append({
        'Vol_Assumption': vol_assumption*100,
        'Vol_Error': (vol_assumption - true_vol)*100,
        'Mean_PnL': mean_pnl,
        'Hedging_Error': hedging_error
    })

vol_df = pd.DataFrame(vol_results)

# Plot volatility assumption analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

ax1.plot(vol_df['Vol_Assumption'], vol_df['Hedging_Error'], 'co-', linewidth=2, markersize=6)
ax1.axvline(x=true_vol*100, color='red', linestyle='--', linewidth=2, label='True Volatility')
ax1.set_xlabel('Volatility Assumption (%)')
ax1.set_ylabel('Hedging Error ($)')
ax1.set_title('Hedging Error vs Volatility Assumption')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(vol_df['Vol_Error'], vol_df['Mean_PnL'], 'yo-', linewidth=2, markersize=6)
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.8)
ax2.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Correct Assumption')
ax2.set_xlabel('Volatility Error (Assumption - True) (%)')
ax2.set_ylabel('Mean P&L ($)')
ax2.set_title('Mean P&L vs Volatility Assumption Error')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nVolatility Assumption Analysis (sample):")
print(vol_df[::2].round(4))  # Show every other row

## 10. Summary and Conclusions

Let's summarize our key findings from this comprehensive options pricing and hedging analysis.

In [ ]:
print("=" * 80)
print("COMPREHENSIVE ANALYSIS SUMMARY")
print("=" * 80)

print("\n🎯 PROJECT OBJECTIVES ACHIEVED:")
print("   ✅ Implemented Black-Scholes pricing with analytical Greeks")
print("   ✅ Developed Monte Carlo pricing engine")
print("   ✅ Built dynamic delta hedging simulator")
print("   ✅ Incorporated stochastic volatility (Heston model)")
print("   ✅ Analyzed impact of transaction costs")
print("   ✅ Performed comprehensive sensitivity analysis")

print("\n📊 KEY FINDINGS:")
improvement = (hedging_error_static - hedging_error_dynamic) / hedging_error_static * 100
print(f"   • Dynamic hedging reduces risk by {improvement:.1f}% vs static hedging")
print(f"   • Hedging error (dynamic): ${hedging_error_dynamic:.2f} ({hedging_error_dynamic/bs_price*100:.1f}% of option value)")
print(f"   • Mean P&L close to zero: ${mean_pnl_dynamic:.3f} (nearly unbiased)")
print(f"   • Transaction costs increase hedging error significantly")
print(f"   • Higher rebalancing frequency improves hedging (up to a point)")
print(f"   • Model risk: Wrong volatility assumption creates systematic bias")

print("\n🧮 THEORETICAL INSIGHTS:")
print("   • Perfect replication is impossible with discrete rebalancing")
print("   • Gamma risk creates P&L volatility between rebalances")
print("   • Stochastic volatility introduces additional model risk")
print("   • Transaction costs create trade-off: more hedging vs higher costs")

print("\n🏢 PRACTICAL APPLICATIONS:")
print("   • Options market makers use similar techniques for risk management")
print("   • Banks hedge derivative portfolios using delta-gamma approaches")
print("   • Fund managers use options for portfolio protection")
print("   • Quantitative analysts optimize rebalancing frequencies")

print("\n🔬 TECHNICAL ACHIEVEMENTS:")
print("   • Modular, extensible codebase")
print("   • Efficient Monte Carlo implementation")
print("   • Comprehensive visualization suite")
print("   • Statistical validation of results")

print("\n🚀 POTENTIAL EXTENSIONS:")
print("   • Add gamma hedging (delta-gamma neutral portfolios)")
print("   • Implement jump-diffusion models")
print("   • Add American option pricing")
print("   • Include dividend yield effects")
print("   • Optimize rebalancing using machine learning")
print("   • Add portfolio of options analysis")

print("\n" + "=" * 80)
print("Thank you for exploring this comprehensive options pricing project!")
print("This demonstrates the intersection of mathematical finance,")
print("computational methods, and practical risk management.")
print("=" * 80)